# First fermionic DMRG

Created 03/01/2025

Objectives:
* Construct the 1D fermionic spt states from the Wang & Gu paper, first pass. Work out the case with trivial projective representation first.
* Not sure of anyway to test the states...? Apart from running the SPT classification on them.

# Imports

In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [13]:
from itertools import combinations

In [14]:
import tenpy
import tenpy.linalg.np_conserved as npc
from tenpy.algorithms import dmrg
from tenpy.networks.mps import MPS

In [15]:
from tenpy.networks.site import ClockSite, FermionSite
from tenpy.models.lattice import Chain
from tenpy.models.model import CouplingModel, NearestNeighborModel, MPOModel, CouplingMPOModel

In [16]:
def mod_4_to_bit(x):    
    possible_mod_4_values = list(range(4))
    assert x in possible_mod_4_values

    x0 = x % 2
    x1 = ((x - x0) // 2) % 2

    return (x0, x1)

In [17]:
def bit_to_mod_4(bit_pair):
    x0, x1 = bit_pair

    possible_mod_2_values = list(range(2))
    assert x0 in possible_mod_2_values
    assert x1 in possible_mod_2_values

    return x0 + 2*x1

In [18]:
def mod_4_to_bit_addition(x, y):
    (x0, x1) = mod_4_to_bit(x)
    (y0, y1) = mod_4_to_bit(y)

    out0 = (x0 + y0) % 2
    out1 = (x1 + y1) % 2

    out = bit_to_mod_4((out0, out1))

    return out

In [19]:
def get_n1_func(n1_01, n1_10):
    l = [0, n1_01, n1_10, n1_01 + n1_10]
    
    def f(g1, g2):
        i = mod_4_to_bit_addition(g1, g2)
        return l[i]

    return f

In [20]:
n1_pair_to_fermion_operators_dict = {
    (0, 0): "Id",
    (0, 1): "C",
    (1, 0): "Cd",
    (1, 1): "N"
}

In [21]:
def get_op_list(g_quad, n1_func):
    out = list()
    g_left, g_in, g_out, g_right = g_quad

    out.append((f'map_{g_left}_{g_left}', 0, 0))

    left_fermion_op_string = n1_pair_to_fermion_operators_dict[
        (n1_func(g_left, g_in), n1_func(g_left, g_out))
    ]

    out.append((left_fermion_op_string, 0, 1))
    
    out.append((f'map_{g_in}_{g_out}', 1, 0))

    right_fermion_op_string = n1_pair_to_fermion_operators_dict[
        (n1_func(g_in, g_right), n1_func(g_out, g_right))
    ]

    out.append((right_fermion_op_string, 1, 1))
    
    out.append((f'map_{g_right}_{g_right}', 2, 0))

    return out

In [22]:
class ClusterIsingFermion(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            ferm = FermionSite(conserve=None)
            sites = [spin, ferm]
            return [spin, ferm], ['s', 'f']

        def init_terms(self, model_params):
            # Read off model parameters
            n1_01, n1_10 = model_params.get('n1', (0,0))
            n1_func = get_n1_func(n1_01, n1_10)
            
            group_quads = combinations(range(4), 4)

            for group_quad in group_quads:
                op_list = get_op_list(group_quad, n1_func)
                self.add_multi_coupling(-1/4, op_list)

In [23]:
M = ClusterIsingFermion({'L': 20})

In [32]:
n1_pairs = [
    (0, 0),
    (0, 1),
    (1, 0),
    (1, 1)
]

In [33]:
models = [
    ClusterIsingFermion({'L': 20, 'n1': p}) for p in n1_pairs
]

In [34]:
psis = [
    MPS.from_lat_product_state(m.lat, [['0', 'empty'],]*20)
    for m in models
]

In [35]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [36]:
for psi in psis:
    psi.canonical_form()

In [37]:
outpsis = list()
energies = list()

for psi, m in zip(psis, models):
    eng = dmrg.TwoSiteDMRGEngine(psi, m, dmrg_params)
    e, psi_out = eng.run()

    outpsis.append(psi_out)
    energies.append(e)

/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/pytho

In [39]:
psis[0]

In [40]:
psis[0].get_B(0)

<npc.Array shape=(1, 4, 1) labels=['vL', 'p', 'vR']>

In [38]:
energies

[0.0, 0.0, 0.0, 0.0]

In [30]:
eng = dmrg.TwoSiteDMRGEngine(psi, M, dmrg_params)
e, psi_out = eng.run()

In [31]:
e

0.0

In [64]:
psi

In [10]:
B_params = np.linspace(1.4, 2, 7)

In [11]:
B_params

array([1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. ])

In [12]:
import h5py
from tenpy.tools import hdf5_io

In [13]:
wavefunctions = list()
energies = list()

for B in B_params:
    print("Commencing B={:.2f} model".format(B))
    model=ClusterIsing({'B': B, 'L':200})
    
    psi = MPS.from_desired_bond_dimension(model.lat.mps_sites(), 1, bc=model.lat.bc_MPS)
    psi.canonical_form()

    eng = dmrg.TwoSiteDMRGEngine(psi, model, dmrg_params)
    e, psi = eng.run()

    wavefunctions.append(psi)
    energies.append(e)

    data = {
        "wavefunction": psi,
        "energy": e,
        "paramters": {"B": B}
    }

    filename = r"data/transverse_cluster_200_site_dmrg_e/{:.2f}".format(B).replace(".", "_")
    filename += ".h5"
    with h5py.File(filename, 'w') as f:
        hdf5_io.save_to_hdf5(f, data)

Commencing B=1.40 model


final DMRG state not in canonical form up to norm_tol=1.00e-05: norm_err=2.80e-05


Commencing B=1.50 model


/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv/lib/python3.12/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
final DMRG state not in canonical form up to norm_tol=1.00e-05: norm_err=1.70e-05


Commencing B=1.60 model


...

/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv/lib/python3.12/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


Commencing B=1.90 model


/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv/lib/python3.12/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


Commencing B=2.00 model


/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv/lib/python3.12/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
